### Hybrid Search Langchain

In [5]:
!pip install --upgrade --quiet  pinecone pinecone-text pinecone-notebooks

In [6]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv("PINECONE_API_KEY")

In [7]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [8]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

c:\Users\vulla\Downloads\CrewAIDemo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
index=pc.Index(index_name)
index

In [10]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
from pinecone_text.sparse import BM25Encoder #this uses tf-idf

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vulla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [12]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans"
]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 61.06it/s]


In [13]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [14]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000017BF5D45570>, index=<pinecone.data.index.Index object at 0x0000017BB9B39DB0>)

In [18]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
]
)

100%|██████████| 1/1 [00:00<00:00,  8.38it/s]


In [23]:
results=retriever.invoke("What city did i visit first")
results

[Document(metadata={'score': 0.240048274}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.233061716}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.212645739}, page_content='In 2023, I visited Paris')]

In [26]:
results[0].page_content

'In 2021, I visited New Orleans'